In [1]:
from dotenv import load_dotenv
import os
load_dotenv(".env")

True

In [2]:
import psycopg2
from psycopg2 import sql
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"), 
    dbname=os.getenv("DB_NAME"), 
    user=os.getenv("DB_USER"), 
    password=os.getenv("DB_PASSWORD"), 
)
cur = conn.cursor()

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import pandas as pd
import re
import os
# Thêm import cần thiết cho tính toán Euclid
import torch.nn.functional as F

# Đặt giới hạn an toàn cho PhoBERT-base
SAFE_MAX_LENGTH = 256 # Giới hạn 256 token là an toàn nhất cho PhoBERT-base
QA_MAX_LENGTH = 256

# Hàm nhúng (đã cập nhật)
def get_embedding(text, model, tokenizer, max_len=512):
    # Đảm bảo tensor luôn ở device của model
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
    with torch.no_grad():
        out = model(**inputs)
    return out.last_hidden_state[:, 0, :] # CLS token

# Hàm tính khoảng cách Euclid giữa một vector và nhiều vector
def euclidean_distance(query_emb, context_embeddings):
    # Mở rộng query_emb để có kích thước [n_chunks, hidden_size]
    query_expanded = query_emb.expand_as(context_embeddings)
    
    # Tính hiệu số, sau đó tính chuẩn L2 (Euclidean norm) trên dimension vector (dim=1)
    distances = torch.norm(context_embeddings - query_expanded, p=2, dim=1)
    
    return distances.squeeze() # Trả về tensor 1D chứa các khoảng cách

model_name = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
phobert = AutoModel.from_pretrained(model_name, trust_remote_code=True)
try: 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    phobert.to(device)
    print(f"Model đang chạy trên thiết bị: {device}")
except Exception as e:
    print(f"Cảnh báo: Không thể chuyển model sang GPU: {e}")
    phobert.to("cpu")

csv_path = "training_input/question.csv"
df = pd.read_csv(csv_path)

for idx, row in df.iterrows():
    question = str(row["Question"]).strip()
    options = [str(row["A"]), str(row["B"]), str(row["C"]), str(row["D"])]
    labels = ["A", "B", "C", "D"]
    
    question_embedding = get_embedding(question, phobert, tokenizer, max_len=512)
    
    if question_embedding is None:
        continue # Bỏ qua câu hỏi rỗng
        
    # Truy vấn DB (không thay đổi)
    cur.execute("""SELECT document FROM dbvector1 
                 Order by embedding  <-> %s :: vector asc
                 LIMIT 1;""", (question_embedding.squeeze().detach().cpu().numpy().tolist(),))
    results = cur.fetchall()
    x = results[0]
    document_name = x[0]
    print(f"\n Câu hỏi: {question}")
   
    # Đọc và chia đoạn tài liệu (logic chia theo 256 ký tự được giữ nguyên)
    md_path = os.path.join("training_output", document_name, "main.md")
    if not os.path.exists(md_path): continue
    
    with open(md_path, "r", encoding="utf-8") as f:
        doc_text = f.read()

    
    processed_text = re.sub(r'\s+', '', doc_text.strip()) 
    doc_chunks = []
    MAX_CHARS = 300 # Giữ 300 ký tự
    for i in range(0, len(processed_text), MAX_CHARS):
        chunk = processed_text[i:i + MAX_CHARS]
        doc_chunks.append(chunk)

    # Tạo embedding cho từng đoạn
    chunk_embeddings = []
    for chunk in doc_chunks:
        # 💥 FIX: Đặt max_length an toàn cho chunk
        emb = get_embedding(chunk, phobert, tokenizer, max_len=300)
        if emb is not None:
            chunk_embeddings.append(emb)

    if not chunk_embeddings:
        print(f"Không tìm thấy chunk hợp lệ trong tài liệu {document_name}. Bỏ qua.")
        continue

    chunk_embeddings = torch.cat(chunk_embeddings, dim=0)  # [n_chunks, hidden_size]

    distances = []
    for i, opt in enumerate(options):
        # *Tối ưu hóa: Nhúng chỉ tùy chọn, so sánh với tất cả các đoạn.*
        qa_text = f"{question} {opt}"
        
       
        qa_emb = get_embedding(qa_text, phobert, tokenizer, max_len=300)

        if qa_emb is None:
            min_dist = float('inf')
        else:
            # THAY ĐỔI LỚN: Tính khoảng cách Euclid (L2 Distance)
            dists = euclidean_distance(qa_emb, chunk_embeddings)
            
            # Tìm khoảng cách NHỎ NHẤT (vì khoảng cách Euclid: Càng nhỏ càng tốt)
            min_simi = dists.min().item()
            
        distances.append(min_simi)
        print(f"  {labels[i]}. {opt}  →  similarity = {min_simi:.4f}")

    # THAY ĐỔI LỚN: Tìm chỉ mục có khoảng cách NHỎ NHẤT (argmin)
    best_idx = torch.argmin(torch.tensor(distances)).item()
    print(f"Đáp án gợi ý: {labels[best_idx]} - {options[best_idx]}")
    print(document_name)

    



Model đang chạy trên thiết bị: cuda

 Câu hỏi: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?
  A. Lưu trữ dữ liệu trên máy chủ  →  similarity = 9.8516
  B. Kết nối Internet và quản lý thiết bị từ xa  →  similarity = 9.5775
  C. Cung cấp dịch vụ phân tích dữ liệu lớn  →  similarity = 10.6536
  D. Thay thế hoàn toàn điện toán đám mây  →  similarity = 9.8783
Đáp án gợi ý: B - Kết nối Internet và quản lý thiết bị từ xa
Public001

 Câu hỏi: Năm 2010, công nghệ nào thường được sử dụng trong kiểm soát ra vào ngôi nhà thông minh?
  A. Bluetooth Low Energy  →  similarity = 10.9916
  B. RFID và nhận dạng khuôn mặt/vân tay  →  similarity = 9.3244
  C. Zigbee  →  similarity = 10.1951
  D. Blockchain  →  similarity = 10.1562
Đáp án gợi ý: B - RFID và nhận dạng khuôn mặt/vân tay
Public001

 Câu hỏi: Bộ cảm biến trong nhà thông minh thực hiện chức năng gì và bộ truyền động có vai trò gì?
  A. Bộ cảm biến hiển thị dữ liệu, bộ truyền động thu thập dữ liệu  →  similarity = 8.3637
  B. Bộ cả

In [ ]:
# ...existing code...
import torch
from transformers import AutoModel, AutoTokenizer
from torch.nn.functional import cosine_similarity
import pandas as pd
import re
import os
# Thêm import cần thiết cho tính toán Euclid
import torch.nn.functional as F

# Đặt giới hạn an toàn cho PhoBERT-base
SAFE_MAX_LENGTH = 256 # Giới hạn 256 token là an toàn nhất cho PhoBERT-base
QA_MAX_LENGTH = 256

# Hàm nhúng (đã cập nhật)
def get_embedding(text, model, tokenizer, max_len=512):
    # Đảm bảo tensor luôn ở device của model
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
    with torch.no_grad():
        out = model(**inputs)
    return out.last_hidden_state[:, 0, :] # CLS token

# Hàm tính khoảng cách Euclid giữa một vector và nhiều vector
def euclidean_distance(query_emb, context_embeddings):
    # Mở rộng query_emb để có kích thước [n_chunks, hidden_size]
    query_expanded = query_emb.expand_as(context_embeddings)
    
    # Tính hiệu số, sau đó tính chuẩn L2 (Euclidean norm) trên dimension vector (dim=1)
    distances = torch.norm(context_embeddings - query_expanded, p=2, dim=1)
    
    return distances.squeeze() # Trả về tensor 1D chứa các khoảng cách

# Hàm shingle theo từ: tạo các cửa sổ trượt theo số từ, mỗi shingle <= max_chars ký tự
def k_shingles(text, k=10, max_chars=300):
    """
    Tạo k-shingles (cửa sổ trượt theo số từ cố định k).
    Nếu tổng số từ < k thì trả về toàn bộ câu như 1 shingle.
    Bỏ qua shingle có độ dài ký tự > max_chars (nếu cần giới hạn).
    """
    words = text.split()
    n = len(words)
    if n == 0:
        return []
    if n < k:
        sh = " ".join(words)
        return [sh] if len(sh) <= max_chars else []
    shingles = []
    for i in range(0, n - k + 1):
        sh = " ".join(words[i:i + k])
        if max_chars is None or len(sh) <= max_chars:
            shingles.append(sh)
    return shingles

model_name = "vinai/phobert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
phobert = AutoModel.from_pretrained(model_name, trust_remote_code=True)
try: 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    phobert.to(device)
    print(f"Model đang chạy trên thiết bị: {device}")
except Exception as e:
    print(f"Cảnh báo: Không thể chuyển model sang GPU: {e}")
    phobert.to("cpu")

csv_path = "training_input/question.csv"
df = pd.read_csv(csv_path)

for idx, row in df.iterrows():
    question = str(row["Question"]).strip()
    options = [str(row["A"]), str(row["B"]), str(row["C"]), str(row["D"])]
    labels = ["A", "B", "C", "D"]
    
    if not question:
        continue

    # Tạo embedding tổng cho câu hỏi nếu cần để tìm document (giữ nguyên logic hiện tại)
    question_embedding = get_embedding(question, phobert, tokenizer, max_len=512)
    if question_embedding is None:
        continue # Bỏ qua câu hỏi rỗng
        
    # Truy vấn DB (không thay đổi)
    cur.execute("""SELECT document FROM dbvector1 
                 Order by embedding  <-> %s :: vector asc
                 LIMIT 1;""", (question_embedding.squeeze().detach().cpu().numpy().tolist(),))
    results = cur.fetchall()
    if not results:
        print("Không tìm thấy document phù hợp trong DB.")
        continue
    x = results[0]
    document_name = x[0]
    print(f"\n Câu hỏi: {question}")

    # Đọc và chuẩn hóa tài liệu
    md_path = os.path.join("training_output", document_name, "main.md")
    if not os.path.exists(md_path):
        print(f"Không tìm thấy file: {md_path}. Bỏ qua.")
        continue
    
    with open(md_path, "r", encoding="utf-8") as f:
        doc_text = f.read()

    processed_text = re.sub(r'\s+', ' ', doc_text.strip())  # giữ khoảng trắng đơn cho tách từ
    doc_chunks = []
    MAX_CHARS = 300 # Giữ 300 ký tự cho chunk tài liệu
    # Chia tài liệu thành các chunk liên tiếp (giữ nguyên logic trước)
    for i in range(0, len(processed_text), MAX_CHARS):
        chunk = processed_text[i:i + MAX_CHARS]
        doc_chunks.append(chunk)

    # Tạo embedding cho từng chunk tài liệu
    chunk_embeddings = []
    for chunk in doc_chunks:
        emb = get_embedding(chunk, phobert, tokenizer, max_len=300)
        if emb is not None:
            chunk_embeddings.append(emb)
    if not chunk_embeddings:
        print(f"Không tìm thấy chunk hợp lệ trong tài liệu {document_name}. Bỏ qua.")
        continue
    chunk_embeddings = torch.cat(chunk_embeddings, dim=0)  # [n_chunks, hidden_size]

    # --- Bây giờ: shingle câu hỏi theo từ, kết hợp với mỗi lựa chọn và tìm khoảng cách nhỏ nhất ---
    question_shingles = k_shingles(question, k=10, max_chars=300)
    if not question_shingles:
        question_shingles = [question]  # fallback

    distances = []
    for i, opt in enumerate(options):
        # Với mỗi lựa chọn, tính min distance trên tất cả các shingle của câu hỏi
        min_dist_for_option = float('inf')
        for sh in question_shingles:
            qa_text = f"{sh} {opt}".strip()
            qa_emb = get_embedding(qa_text, phobert, tokenizer, max_len=300)
            if qa_emb is None:
                continue
            # Tính khoảng cách Euclid tới tất cả chunk và lấy min
            dists = euclidean_distance(qa_emb, chunk_embeddings)
            min_d = dists.min().item()
            if min_d < min_dist_for_option:
                min_dist_for_option = min_d
        if min_dist_for_option == float('inf'):
            min_dist_for_option = 1e9
        distances.append(min_dist_for_option)
        print(f"  {labels[i]}. {opt}  →  min_distance_over_shingles = {min_dist_for_option:.4f}")

    # Chọn đáp án có khoảng cách nhỏ nhất (gần nhất)
    best_idx = int(torch.argmin(torch.tensor(distances)).item())
    print(f"Đáp án gợi ý: {labels[best_idx]} - {options[best_idx]}")
    print(document_name)
# ...existing code...

Model đang chạy trên thiết bị: cuda

 Câu hỏi: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?
  A. Lưu trữ dữ liệu trên máy chủ  →  min_distance_over_shingles = 7.2856
  B. Kết nối Internet và quản lý thiết bị từ xa  →  min_distance_over_shingles = 7.1428
  C. Cung cấp dịch vụ phân tích dữ liệu lớn  →  min_distance_over_shingles = 7.8296
  D. Thay thế hoàn toàn điện toán đám mây  →  min_distance_over_shingles = 7.3177
Đáp án gợi ý: B - Kết nối Internet và quản lý thiết bị từ xa
Public001

 Câu hỏi: Năm 2010, công nghệ nào thường được sử dụng trong kiểm soát ra vào ngôi nhà thông minh?
  A. Bluetooth Low Energy  →  min_distance_over_shingles = 8.8424
  B. RFID và nhận dạng khuôn mặt/vân tay  →  min_distance_over_shingles = 8.3491
  C. Zigbee  →  min_distance_over_shingles = 8.7251
  D. Blockchain  →  min_distance_over_shingles = 8.5748
Đáp án gợi ý: B - RFID và nhận dạng khuôn mặt/vân tay
Public001

 Câu hỏi: Bộ cảm biến trong nhà thông minh thực hiện chức năng gì và bộ truy

In [ ]:
cur.close()
conn.close()

In [64]:
conn.rollback()